## Load and processes result files generated from all proposed models. The results include similarity scores of all images per new article in the test dataset.

In [1]:
lines_caption_sim = []
with open('caption_sim_wmd.txt') as f:
    lines_caption_sim = f.readlines()

In [2]:
image_train_sim = []
with open('train_img_similarity.txt') as f:
    image_train_sim = f.readlines()

In [3]:
image_crawl_sim = []
with open('crawl_img_similarity.txt') as f:
    image_crawl_sim = f.readlines()

In [4]:
ar_cap_sim_dic={}
for line in lines_caption_sim:
    segs=line.strip().split("\t")
    if segs[0] not in ar_cap_sim_dic:
        ar_cap_sim_dic[segs[0]]={}
    ar_cap_sim_dic[segs[0]][segs[1]]= 1-float(segs[2])

In [5]:
len(ar_cap_sim_dic)

1915

In [6]:
ar_train_sim_dic={}
for line in image_train_sim:
    segs=line.strip().split("\t")
    if segs[0] not in ar_train_sim_dic:
        ar_train_sim_dic[segs[0]]=[]
    ar_train_sim_dic[segs[0]].append((segs[1], segs[2], segs[3]))

In [7]:
ar_train_sim_dic_cal={}
for k, v in ar_train_sim_dic.items():
    if k not in ar_train_sim_dic_cal:
        ar_train_sim_dic_cal[k]={}
    for item in v:
        if int(item[2])==0:
            sim=0
        else:
            sim=(1-float(item[1]))*int(item[2])
        ar_train_sim_dic_cal[k][item[0]]= sim

In [8]:
ar_crawl_sim_dic={}
for line in image_crawl_sim:
    segs=line.strip().split("\t")
    if segs[0] not in ar_train_sim_dic:
        ar_crawl_sim_dic[segs[0]]=[]
    ar_crawl_sim_dic[segs[0]].append((segs[1], segs[2], segs[3]))

In [9]:
ar_crawl_sim_dic_cal={}
for k, v in ar_crawl_sim_dic.items():
    if k not in ar_train_sim_dic_cal:
        ar_crawl_sim_dic_cal[k]={}
    for item in v:
        if int(item[2])==0:
            sim=0
        else:
            sim=(1-float(item[1]))*int(item[2])
        ar_crawl_sim_dic_cal[k][item[0]]= sim

In [10]:
def norm_dict(a_dict):
    result={}
    amin, amax = min(a_dict.values()), max(a_dict.values())
    for k, v in a_dict.items():
        if amax-amin==0:
            result[k]=0
        else:
            result[k] = (v-amin) / (amax-amin)
    return result

In [11]:
sample_dict={1:1, 2:2,3:3, 4:4}
print(norm_dict(sample_dict))

{1: 0.0, 2: 0.3333333333333333, 3: 0.6666666666666666, 4: 1.0}


In [12]:
#ar_cap_sim_dic, ar_train_sim_dic_cal, ar_crawl_sim_dic_cal
def norm_sim(a_dict):
    result={}
    print(len(a_dict))
    for k, v in a_dict.items():
        result[k]=norm_dict(v)
    return result

In [13]:
ar_cap_sim_norm=norm_sim(ar_cap_sim_dic)

1915


In [14]:
ar_train_sim_norm={}
ar_train_sim_norm=norm_sim(ar_train_sim_dic_cal)

128


In [15]:
print(len(ar_train_sim_norm))

128


In [16]:
ar_crawl_sim_norm={}
ar_crawl_sim_norm=norm_sim(ar_crawl_sim_dic_cal)

191


In [17]:
print(len(ar_crawl_sim_norm))

191


In [26]:
import os
def merge_all(cap_dict, train_dict, crawl_dict, weight_cap, weight_img):
    result={}
    for k, v in cap_dict.items():
        img_id=os.path.splitext(k)[0]
        result[img_id]=v
        if k in train_dict:
            for k_tr, v_tr in train_dict[k].items():
                result[img_id][k_tr]=v_tr*weight_img
        if k in crawl_dict:
            for k_cr, v_cr in crawl_dict[k].items():
                result[img_id][k_cr]=v_cr*weight_img
    return result

In [27]:
result = merge_all(ar_cap_sim_dic, ar_train_sim_norm, ar_crawl_sim_norm, 0, 1)

In [28]:
print(len(result))

1915


In [29]:
final_result=norm_sim(result)

1915


In [30]:
def sort_dict(input):
    result={}
    for k, v in input.items():
        sort_v=dict(sorted(v.items(), key=lambda item: item[1], reverse=True))
        result[k]=sort_v
    return result

In [31]:
sort_final_result=sort_dict(final_result)

In [33]:
f = open("final_final_result_1115_run2.txt", "a")
for key, value in sort_final_result.items():
    result=key
    for k,v in value.items():
        result+="\t"+k+"\t"+str(v)
    f.write(result+"\n")
f.close()

In [28]:
with open('all_results_by_articleID_and_imageID_with_similarity_score_andres.txt') as f:
    next(f)
    class_lines = f.readlines()

In [29]:
class_dict={}
for line in class_lines:
    line=line.strip()
    segs=line.split(",")
    if segs[0] not in class_dict:
        class_dict[segs[0]]={}
    for i in range(1, len(segs)):
        if len(segs[i])>0:
            img_id=segs[i].strip().split(" ")[0]
            score=segs[i].strip().split(" ")[1].replace('(','').replace(')','')
            class_dict[segs[0]][img_id]=float(score)

In [30]:
class_result_final=norm_sim(class_dict)

1910


In [39]:
def combine_all(sort_final_result, class_result_final):
    result_comb={}
    cnt=0
    for key, value in sort_final_result.items():
        result_comb[key]={}
        for cap_k, cap_v in value.items():
            img_id=os.path.splitext(cap_k)[0]
            result_comb[key][img_id]=cap_v        
        if key in class_dict:
            for cap_k_in, cap_v_in in result_comb[key].items():
                if cap_k_in in class_result_final[key]:
                    cnt+=1
                    sum=float(cap_v_in)+float(class_result_final[key][cap_k_in])
                    result_comb[key][cap_k_in]=sum
    print(cnt)
    return result_comb

In [40]:
final_result=combine_all(sort_final_result, class_result_final)

3625180


In [42]:
sort_final_result=sort_dict(final_result)

In [45]:
f = open("final_combine_result.txt", "a")
for key, value in sort_final_result.items():
    result=key
    for k,v in value.items():
        result+="\t"+k+":"+str(v)
    f.write(result+"\n")
f.close()

In [52]:
f = open("submission.tsv", "a")
header="particleID"
for i in range(100):
    header+="\t"+"iid"+str(i+1)
f.write(header+"\n")
for key, value in sort_final_result.items():
    l=[*value]
    l_100 = l[0:100]
    l_join=key+"\t"+'\t'.join([str(x) for x in l_100])
    f.write(l_join+"\n")
f.close()